# Importing library

In [27]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

## loading the data

In [30]:
credit_card_data = pd.read_csv('./creditcard.csv')

In [31]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Distribution of legit transction and fraud transction

In [33]:
credit_card_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

#### this data is highly unbalance
#### legit-->0
#### fraud-->1

In [69]:
legit = credit_card_data[credit_card_data.Class==0]
fraud = credit_card_data[credit_card_data.Class==1]

In [71]:
print(legit)

            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  0.098698  0.363787  ... -0.01830

### stastical measurement

In [74]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [76]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [80]:
credit_card_data.groupby('Class').describe()

Time                                                       \
          count          mean           std    min      25%      50%   
Class                                                                  
0      284315.0  94838.202258  47484.015786    0.0  54230.0  84711.0   
1         492.0  80746.806911  47835.365138  406.0  41241.5  75568.5   

                                 V1            ...       V28             \
            75%       max     count      mean  ...       75%        max   
Class                                          ...                        
0      139333.0  172792.0  284315.0  0.008258  ...  0.077962  33.847808   
1      128483.0  170348.0     492.0 -4.771948  ...  0.381152   1.779364   

         Amount                                                              
          count        mean         std  min   25%    50%     75%       max  
Class                                                                        
0      284315.0   88.291022  250.105092  0.0  5.65  22.00   77.05  25691.16  
1         492.0  122.211321  256.683288  0.0  1.00   9.25  105.89   2125.87  

[2 rows x 240 columns]

##### building a dataset having uniform distribution is that 492 legit transction and 492 fraud transction

In [85]:
legit_sample = legit.sample(n=492)

#### concating the two data frame

In [92]:
new_data_frame = pd.concat([legit_sample,fraud],axis=0)

In [94]:
new_data_frame.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
37805,39119.0,-1.154787,1.221601,0.996147,-0.174806,-0.069490,-0.459002,0.264676,0.409003,-0.760416,...,-0.125167,-0.489680,0.085510,0.000777,-0.127222,0.052459,-0.123397,0.056327,10.99,0
163358,115868.0,1.527933,-1.746153,-1.268955,-1.867807,-1.024196,-0.722644,-0.269411,-0.157429,2.603405,...,0.402589,0.756221,-0.145311,0.640017,-0.061427,-0.796857,0.019242,0.018141,294.79,0
135575,81314.0,-1.956290,1.984792,0.019815,-0.014324,-0.998977,-0.519365,-0.611070,1.565012,-0.604449,...,-0.124105,-0.608872,0.216471,0.131101,-0.160423,0.092211,0.103626,-0.005673,14.99,0
205477,135762.0,2.042203,0.142931,-1.614000,1.400356,0.294700,-1.189642,0.528599,-0.329204,0.438596,...,0.027428,0.250704,0.023571,-0.050713,0.408136,-0.483441,-0.023406,-0.066691,1.00,0
27987,34801.0,1.384715,-0.716893,0.150579,-0.616782,-0.798662,-0.325054,-0.579571,-0.089192,-0.305863,...,-0.080672,-0.235436,-0.142452,-0.442976,0.608156,-0.220251,0.005828,0.010857,43.29,0


In [96]:
new_data_frame['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [98]:
new_data_frame.groupby('Class').describe()


Time                                                                 \
       count          mean           std    min      25%      50%       75%   
Class                                                                         
0      492.0  93261.725610  47890.151269  365.0  50956.0  82703.5  138498.5   
1      492.0  80746.806911  47835.365138  406.0  41241.5  75568.5  128483.0   

                    V1            ...       V28           Amount              \
            max  count      mean  ...       75%       max  count        mean   
Class                             ...                                          
0      172267.0  492.0 -0.041613  ...  0.075937  2.569189  492.0   92.847703   
1      170348.0  492.0 -4.771948  ...  0.381152  1.779364  492.0  122.211321   

                                                       
              std  min  25%    50%       75%      max  
Class                                                  
0      215.631296  0.0  5.0  20.53   82.4225  2237.64  
1      256.683288  0.0  1.0   9.25  105.8900  2125.87  

[2 rows x 240 columns]

In [110]:
X = new_data_frame.drop(columns='Class',axis=1)
y = new_data_frame['Class']

In [112]:
print(y)

37805     0
163358    0
135575    0
205477    0
27987     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [116]:
print(X)


            Time        V1        V2        V3        V4        V5        V6  \
37805    39119.0 -1.154787  1.221601  0.996147 -0.174806 -0.069490 -0.459002   
163358  115868.0  1.527933 -1.746153 -1.268955 -1.867807 -1.024196 -0.722644   
135575   81314.0 -1.956290  1.984792  0.019815 -0.014324 -0.998977 -0.519365   
205477  135762.0  2.042203  0.142931 -1.614000  1.400356  0.294700 -1.189642   
27987    34801.0  1.384715 -0.716893  0.150579 -0.616782 -0.798662 -0.325054   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

#### splitting the data into Training and test data

In [121]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [123]:
print(X.shape,X_train.shape,X_test.shape)

(984, 30) (787, 30) (197, 30)


#### Model training using Logistic regression 

In [126]:
model  =LogisticRegression()

In [130]:
model.fit(X_train,y_train)

LogisticRegression()

#### Model Evaluation

In [148]:
train_predict = model.predict(X_train)

### Model evaluation

In [151]:
train_accuracy = accuracy_score(train_predict,y_train)

In [153]:
print("accuracy on training data : ",train_accuracy)

accuracy on training data :  0.9415501905972046


In [155]:
test_predict = model.predict(X_test)
test_accuracy = accuracy_score(test_predict,y_test)
print("accuracy on test data",test_accuracy)

accuracy on test data 0.9238578680203046
